In [16]:
import arviz as az
import torch
import numpy as np
import camb
from tqdm import tqdm

from sbi.inference import MNLE, likelihood_estimator_based_potential, SNLE
from pyro.distributions import InverseGamma
from torch.distributions import Beta, Binomial, Gamma
from sbi import utils as utils

from sbi.inference import MCMCPosterior

# Seeding
torch.manual_seed(1);

In [17]:
def sim(params):
    # Create a set of cosmological parameters
    pars = camb.CAMBparams()
    H0 = params[0] * 100
    pars.set_cosmology(H0=H0)
    pars.set_for_lmax(2500, lens_potential_accuracy=0)
    results = camb.get_results(pars)
    powers = results.get_cmb_power_spectra(pars, CMB_unit='muK')
    total = powers['total']
    cl = total[:, 0]
    return cl[2:]

In [6]:
parameter_min = [50, 0.02, 0.1, 2.8e-1, 0.9]
parameter_max = [90, 0.025, 0.15, 3.2e-1, 1.0]

# Define prior for all parameters
prior = utils.BoxUniform(low=torch.as_tensor(parameter_min),
                         high=torch.as_tensor(parameter_max))

In [9]:
trainer = SNLE(prior)
theta = torch.tensor(np.load('params.npy')[:10000], dtype = torch.float32)
x = torch.tensor(np.load('cls.npy')[:10000], dtype = torch.float32)
likelihood_estimator = trainer.append_simulations(theta, x).train()

 Neural network successfully converged after 212 epochs.

In [10]:
import pickle
with open('like_est_mcmc_5d.pkl', 'wb') as file:
    pickle.dump(likelihood_estimator, file)

In [ ]:
import pickle
with open('like_est_mcmc_5d.pkl', 'rb') as file:
    likelihood_estimator = pickle.load( file)

In [13]:
torch.manual_seed(42)
num_trials = 100
theta_o_i = [67.6, 0.0220, 0.122, 3.085e-1, 0.964]
theta_o = [theta_o_i for _ in range(num_trials)]
x_o = [sim([0.676]) for i in theta_o]
np.save('x_o_mcmc_5d.npy', x_o)
x_o = torch.tensor(x_o, dtype = torch.float32)

# Set MCMC parameters and run Pyro NUTS.
mcmc_parameters = dict(
    num_chains=4,
    thin=5,
    warmup_steps=50,
    init_strategy="proposal",
    method="nuts",
)
num_samples = 5000

# get the potential function and parameter transform for constructing the posterior
potential_fn, parameter_transform = likelihood_estimator_based_potential(
    likelihood_estimator, prior, x_o
)
mnle_posterior = MCMCPosterior(
    potential_fn, proposal=prior, theta_transform=parameter_transform, **mcmc_parameters
)

mnle_samples = mnle_posterior.sample(
    (num_samples,), x=x_o, show_progress_bars=False
)
# get arviz InferenceData object from posterior
inference_data = mnle_posterior.get_arviz_inference_data()

/var/folders/b7/8mcnnrs965d05l_d6__pc9fc0000gn/T/ipykernel_56937/1744455396.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/miniforge3/conda-bld/pytorch-recipe_1696859376244/work/torch/csrc/utils/tensor_new.cpp:248.)
  x_o = torch.tensor(x_o, dtype = torch.float32)
/Users/aisulu/anaconda3/envs/myenv/lib/python3.8/site-packages/sbi/utils/sbiutils.py:316: UserWarning: An x with a batch size of 100 was passed. It will be interpreted as a batch of independent and identically
            distributed data X={x_1, ..., x_n}, i.e., data generated based on the
            same underlying (unknown) parameter. The resulting posterior will be with
            respect to entire batch, i.e,. p(theta | X).
  warnings.warn(
2024-03-06 21:51:38.317299: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorF

KeyboardInterrupt: 

In [ ]:
import pickle
with open('inference_data_mcmc.pkl', 'wb') as file:
    pickle.dump(inference_data, file)

In [ ]:
az.style.use("arviz-darkgrid")
az.plot_rank(inference_data)

In [ ]:
az.plot_autocorr(inference_data);

In [ ]:
az.plot_trace(inference_data, compact=False);

In [ ]:
az.plot_posterior(inference_data)

In [ ]:
print(
    f"Given the {num_trials} we observed, the posterior is centered around true underlying parameters theta_o: {theta_o}"
)

In [ ]:
az.plot_pair(inference_data)

In [ ]:
az.plot_ess(inference_data, kind="evolution");

In [ ]:
az.plot_pair(
    inference_data,
    var_names=["theta"],
    kind="hexbin",
    marginals=True,
    figsize=(7, 7),
)